<a href="https://colab.research.google.com/github/AhmetSarici/DataScience/blob/ML/CokluDogrusalRegresyon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Çoklu Doğrusal Regresyon¶


Temel amaç, bağımlı ve bağımsız değişkenler arasındaki ilişkiyi ifade eden doğrusal fonksiyonu bulmak. Yönü ve şiddeti ile ilgili yorumlar çıkarabilmek.

In [ ]:
import pandas as pd
ad = pd.read_csv("Advertising.csv", usecols = [1,2,3,4])
df = ad.copy()
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict

In [ ]:
#Sales değişkenini dışarıda bırakıp tüm bağımsız değişkenleri alma
X = df.drop("sales", axis = 1)
y = df["sales"]

#Train ve test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state= 42)


In [ ]:
X_train.shape

(160, 3)

In [ ]:
y_train.shape

(160,)

In [ ]:
X_test.shape

(40, 3)

In [ ]:
y_test.shape

(40,)

In [ ]:
training = df.copy() #veri setinin bir kopyası

In [ ]:
training.shape

(200, 4)

## Statsmodels

In [ ]:
#Yorumlama için Statsmodel
lm = sm.OLS(y_train, X_train)

In [ ]:
model = lm.fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  sales   R-squared (uncentered):                   0.982
Model:                            OLS   Adj. R-squared (uncentered):              0.982
Method:                 Least Squares   F-statistic:                              2935.
Date:                Tue, 31 May 2022   Prob (F-statistic):                   1.28e-137
Time:                        13:54:02   Log-Likelihood:                         -336.65
No. Observations:                 160   AIC:                                      679.3
Df Residuals:                     157   BIC:                                      688.5
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
TV             0.0531      0.001     36.467      0.000       0.050       0.056
radio          0.2188      0.011     20.138      0.000       0.197       0.240
newspaper      0.0239      0.008      3.011      0.003       0.008       0.040
==============================================================================
Omnibus:                       11.405   Durbin-Watson:                   1.895
Prob(Omnibus):                  0.003   Jarque-Bera (JB):               15.574
Skew:                          -0.432   Prob(JB):                     0.000415
Kurtosis:                       4.261   Cond. No.                         13.5
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

R-suquared = Bağımlı değişkeninin anlamlılığını açıklama oranı

In [ ]:
model.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
TV,0.0531,0.001,36.467,0.000,0.050,0.056
radio,0.2188,0.011,20.138,0.000,0.197,0.240
newspaper,0.0239,0.008,3.011,0.003,0.008,0.040


## scikit-learn model

In [ ]:
lm = LinearRegression()
#ilk argüman bağımsız değişkenler
model = lm.fit(X_train, y_train)

In [ ]:
#sabit katsayı
model.intercept_

2.979067338122629

In [ ]:
#bağımsız değişkenlerin katsayısı
model.coef_

array([0.04472952, 0.18919505, 0.00276111])

## Tahmin

Model denklemi:

Sales = 2.97 + TV0.04 + radio0.18 + newspaper*0.002

Örneğin 30 birim TV harcaması, 10 birim radio harcamasi, 40 birimde gazete harcaması olduğunda satışların tahmini değeri ne olur?



In [ ]:
yeni_veri = [[30],[10],[40]]
#Dataframe dönüştürmemiz gerekiyor
yeni_veri = pd.DataFrame(yeni_veri).T

In [ ]:
model.predict(yeni_veri)

array([6.32334798])

In [ ]:
#Modelin tahmin başarısı nedir?
#y_train
#model.predict(X_train) eğitim hatasını buluyoruz =y^
rmse = np.sqrt(mean_squared_error(y_train, model.predict(X_train)))

In [ ]:
rmse

1.644727765644337

In [ ]:
#Test hatası için
rmse = np.sqrt(mean_squared_error(y_test, model.predict(X_test)))

In [ ]:
rmse

1.78159966153345

## Model Tuning / Model Doğrulama

In [ ]:
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [ ]:
X = df.drop('sales', axis=1)
y = df["sales"]
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.20, 
                                                    random_state=144)
#random_state = modelin belirli kısmını seçiyoruz
lm = LinearRegression() 
model = lm.fit(X_train, y_train)

In [ ]:
np.sqrt(mean_squared_error(y_train, model.predict(X_train)))

1.6748559274650712

In [ ]:
np.sqrt(mean_squared_error(y_test, model.predict(X_test)))

1.6640263686701031

In [ ]:
model.score(X_train, y_train)

0.8971614078663419

In [ ]:
#cv=kaç katlı
#Valide edilmiş hata
cross_val_score(model, X_train, y_train, cv = 10, scoring = "r2").mean()

0.8733783298422942

In [ ]:
#Eğitim için valide edilmiş hata hesaplama
#scoring = "neg_mean_squared_error" pozitif değer oluşturmak için
#Gerçek eğitim hatası
np.sqrt(-cross_val_score(model, 
                X_train, 
                y_train, 
                cv = 10, 
                scoring = "neg_mean_squared_error")).mean()

1.6649345607872932

In [ ]:
#Test için valide edilmiş hata hesaplama
#Gerçek test hatası
np.sqrt(-cross_val_score(model, 
                X_test, 
                y_test, 
                cv = 10, 
                scoring = "neg_mean_squared_error")).mean()

1.7399924960346644